##__Final Project: EDA__

Name: Drew Zink

Topic: NCAA Division 1 College Basketball Predictive Metrics

[*Dataset Origin*](https://www.kaggle.com/datasets/andrewsundberg/college-basketball-dataset)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

For this project, I decided to obtain a dataset from Kaggle pertaining to College Basketball. I have been fascinated with the do-or-die nature of the NCAA Tournament since I first fell in love with sports at the age of 12, so I thought it would be a good fit for me since it is something I am passionate about. This dataset contains statistics from the last 10 years (2013-2023) on every single season a college basketball team has played.

In [3]:
# Load College Basketball Dataset
cbb = pd.read_csv('cbb.csv')
cbb.head(5)

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
0,North Carolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,...,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1.0,2016
1,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,...,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1.0,2015
2,Michigan,B10,40,33,114.4,90.4,0.9375,53.9,47.7,14.0,...,30.0,54.7,46.8,35.2,33.2,65.9,6.9,2ND,3.0,2018
3,Texas Tech,B12,38,31,115.2,85.2,0.9696,53.5,43.0,17.7,...,36.6,52.8,41.9,36.5,29.7,67.5,7.0,2ND,3.0,2019
4,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,...,26.9,56.3,40.0,38.2,29.0,71.5,7.7,2ND,1.0,2017


In [4]:
## Print the number of rows and columns in the DataFrame.
print(f'This dataframe consists of {cbb.shape[0]} rows and {cbb.shape[1]} columns')

This dataframe consists of 3523 rows and 24 columns


In [ ]:
cbb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3523 entries, 0 to 3522
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   TEAM        3523 non-null   object 
 1   CONF        3523 non-null   object 
 2   G           3523 non-null   int64  
 3   W           3523 non-null   int64  
 4   ADJOE       3523 non-null   float64
 5   ADJDE       3523 non-null   float64
 6   BARTHAG     3523 non-null   float64
 7   EFG_O       3523 non-null   float64
 8   EFG_D       3523 non-null   float64
 9   TOR         3523 non-null   float64
 10  TORD        3523 non-null   float64
 11  ORB         3523 non-null   float64
 12  DRB         3523 non-null   float64
 13  FTR         3523 non-null   float64
 14  FTRD        3523 non-null   float64
 15  2P_O        3523 non-null   float64
 16  2P_D        3523 non-null   float64
 17  3P_O        3523 non-null   float64
 18  3P_D        3523 non-null   float64
 19  ADJ_T       3523 non-null  

After performing some preliminary examination of the dataset, this structure is full of potential features I can use. The majority of the dataset consists of numerical features quantifying performance. These include the number of games played and wins obtained, offensive and defensive efficiency per 100 possessions, BARTHAG power ratings, as well as metrics pertaining to shooting, rebounding, free throws, and tempo, all on offense and defense. There are also a few categorical variables present as well, including the names of the teams, the conference they are a member of, and the result of their postseason campaign in the NCAA Tournament. Note that only 68 teams make the tourney, so there are many teams that have no postseason result or seed. To shrink the dataset, I am only going to consider teams that make the tournament.

In [5]:
# Extract all observations where 'SEED' is not nan
cbb = cbb.dropna(subset=['SEED'])
cbb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 680 entries, 0 to 3227
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   TEAM        680 non-null    object 
 1   CONF        680 non-null    object 
 2   G           680 non-null    int64  
 3   W           680 non-null    int64  
 4   ADJOE       680 non-null    float64
 5   ADJDE       680 non-null    float64
 6   BARTHAG     680 non-null    float64
 7   EFG_O       680 non-null    float64
 8   EFG_D       680 non-null    float64
 9   TOR         680 non-null    float64
 10  TORD        680 non-null    float64
 11  ORB         680 non-null    float64
 12  DRB         680 non-null    float64
 13  FTR         680 non-null    float64
 14  FTRD        680 non-null    float64
 15  2P_O        680 non-null    float64
 16  2P_D        680 non-null    float64
 17  3P_O        680 non-null    float64
 18  3P_D        680 non-null    float64
 19  ADJ_T       680 non-null    float

I want to use this project to create a model that could predict whether a team has the potential make it "deep" in the NCAA tournament or not. For a team to meet this criteria in this dataset, they must win **three games**, or make it to the Elite 8. To begin this process, I need to distinguish Power conference teams from Mid-Major teams. Power conference teams have more funding and influence than a Mid-Major conference, so I need to include this as a quantifiable factor in the dataset. The CONF feature contains tags for each conference, so I can use unique() to figure out what the tags are for our Power Conferences.

In [6]:
cbb['CONF'].unique()

array(['ACC', 'B10', 'B12', 'WCC', 'SEC', 'BE', 'Amer', 'P12', 'A10',
       'MVC', 'AE', 'BSth', 'BW', 'CUSA', 'Ivy', 'MAC', 'MWC', 'OVC',
       'SB', 'SC', 'Slnd', 'Sum', 'ASun', 'BSky', 'CAA', 'Horz', 'MAAC',
       'MEAC', 'NEC', 'Pat', 'SWAC', 'WAC'], dtype=object)

In College Football, the Power 5 consists of the Atlantic Coast Conference, the Big Ten, the Big Twelve, the Southeastern Conference, and the Pacific-12. In College Basketball, the Power Conferences include these 5, as well as the Big East and the American Conference. The tags for these conferences are ACC, B10, B12, SEC, P12, BE, and Amer, respectively. We can use this information to make a new feature named P7, which will indicate if a team is a Power Conference team or not.

In [7]:
# Power 7 Conferences are ACC, B10, B12, SEC, BE, P12, and Amer
# Wish to create a column indicating if an entry belongs to P7 or not
cbb['P7'] = cbb['CONF'].isin(['ACC', 'B10', 'B12', 'SEC', 'BE', 'P12', 'Amer'])
cbb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 680 entries, 0 to 3227
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   TEAM        680 non-null    object 
 1   CONF        680 non-null    object 
 2   G           680 non-null    int64  
 3   W           680 non-null    int64  
 4   ADJOE       680 non-null    float64
 5   ADJDE       680 non-null    float64
 6   BARTHAG     680 non-null    float64
 7   EFG_O       680 non-null    float64
 8   EFG_D       680 non-null    float64
 9   TOR         680 non-null    float64
 10  TORD        680 non-null    float64
 11  ORB         680 non-null    float64
 12  DRB         680 non-null    float64
 13  FTR         680 non-null    float64
 14  FTRD        680 non-null    float64
 15  2P_O        680 non-null    float64
 16  2P_D        680 non-null    float64
 17  3P_O        680 non-null    float64
 18  3P_D        680 non-null    float64
 19  ADJ_T       680 non-null    float

Next, I want to make my target feature, DEEP. This feature will indicate whether or not a team made it to the Elite 8 or further in their season. We can use the same method we used to create the P7 feature.

In [8]:
# My target variable for this project is going to be teams "going deep" in the
# NCAA tournament. To go deep, you need to win at least 3 games, or make it to
# the Elite 8, denoted by E8 in the POSTSEASON feature set.
# I am going to create a feature named DEEP that meets this category.
print(cbb['POSTSEASON'].unique())
print(cbb['SEED'].unique())

# Teams denoted as Champion, 2ND, F4, or E8 meet this category.
cbb['DEEP'] = cbb['POSTSEASON'].isin(['Champions', '2ND', 'F4', 'E8'])
cbb.info()

['2ND' 'Champions' 'E8' 'F4' 'R32' 'R64' 'R68' 'S16']
[ 1.  3.  8.  4.  2.  7.  6.  9. 11. 10.  5. 16. 12. 13. 14. 15.]
<class 'pandas.core.frame.DataFrame'>
Index: 680 entries, 0 to 3227
Data columns (total 26 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   TEAM        680 non-null    object 
 1   CONF        680 non-null    object 
 2   G           680 non-null    int64  
 3   W           680 non-null    int64  
 4   ADJOE       680 non-null    float64
 5   ADJDE       680 non-null    float64
 6   BARTHAG     680 non-null    float64
 7   EFG_O       680 non-null    float64
 8   EFG_D       680 non-null    float64
 9   TOR         680 non-null    float64
 10  TORD        680 non-null    float64
 11  ORB         680 non-null    float64
 12  DRB         680 non-null    float64
 13  FTR         680 non-null    float64
 14  FTRD        680 non-null    float64
 15  2P_O        680 non-null    float64
 16  2P_D        680 non-null    float64


I also want to modify the POSTSEASON feature so that it is no longer categorical. To do this, I am going to replace each of the observations present with a number from 0 to 7, with 0 being a team that didn't make the tournament and 7 being a team that won it.

In [9]:
# I also want to transform POSTSEASON into a numerical variable, with teams
# gaining a point for each round they advance to. I am now going to replace
# each of them with a number. Note that I am treating R68 and R64 the same
# since only 8 teams play in the Round of 64.
replace = {
    np.nan : 0,
    'R68': 1,
    'R64': 1,
    'R32': 2,
    'S16': 3,
    'E8': 4,
    'F4': 5,
    '2ND': 6,
    'Champions': 7
}

cbb['POSTSEASON'] = cbb['POSTSEASON'].replace(replace).astype(int)

cbb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 680 entries, 0 to 3227
Data columns (total 26 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   TEAM        680 non-null    object 
 1   CONF        680 non-null    object 
 2   G           680 non-null    int64  
 3   W           680 non-null    int64  
 4   ADJOE       680 non-null    float64
 5   ADJDE       680 non-null    float64
 6   BARTHAG     680 non-null    float64
 7   EFG_O       680 non-null    float64
 8   EFG_D       680 non-null    float64
 9   TOR         680 non-null    float64
 10  TORD        680 non-null    float64
 11  ORB         680 non-null    float64
 12  DRB         680 non-null    float64
 13  FTR         680 non-null    float64
 14  FTRD        680 non-null    float64
 15  2P_O        680 non-null    float64
 16  2P_D        680 non-null    float64
 17  3P_O        680 non-null    float64
 18  3P_D        680 non-null    float64
 19  ADJ_T       680 non-null    float

Lastly, I am going to remove the TEAM, CONF, YEAR, POSTSEASON, and G features from the dataset. Each observation should be treated as a blind resume, with only numbers present. The CONF feature is also no longer needed, as the P7 feature quantifies it. I believe that the YEAR feature doesn't add anything, and only confuses the model since there aren't more teams making it to the Elite 8 or further each year (only 8 teams advance this far every year). I am also removing the POSTSEASON feature because it is not predictive and is accounted for by the target variable DEEP. Finally, I am removing G because I think that W is a similar enough feature, as the more games you play, the more games you win, and the better you will typically do.

In [10]:
# Removing TEAM feature, as supposed to view these as blind resumes
cbb = cbb.drop(['TEAM', 'CONF', 'YEAR', 'POSTSEASON', 'G'], axis=1)

cbb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 680 entries, 0 to 3227
Data columns (total 21 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   W        680 non-null    int64  
 1   ADJOE    680 non-null    float64
 2   ADJDE    680 non-null    float64
 3   BARTHAG  680 non-null    float64
 4   EFG_O    680 non-null    float64
 5   EFG_D    680 non-null    float64
 6   TOR      680 non-null    float64
 7   TORD     680 non-null    float64
 8   ORB      680 non-null    float64
 9   DRB      680 non-null    float64
 10  FTR      680 non-null    float64
 11  FTRD     680 non-null    float64
 12  2P_O     680 non-null    float64
 13  2P_D     680 non-null    float64
 14  3P_O     680 non-null    float64
 15  3P_D     680 non-null    float64
 16  ADJ_T    680 non-null    float64
 17  WAB      680 non-null    float64
 18  SEED     680 non-null    float64
 19  P7       680 non-null    bool   
 20  DEEP     680 non-null    bool   
dtypes: bool(2), float64(

Note that we are now down to 20 features, including the target feature, with these creations. We need to convert all non-float features to a float to make it easier to model. For the boolean cases, 1 will represent True and 0 will represent False.

In [11]:
cbb['W'] = cbb['W'].astype(float)
cbb['SEED'] = cbb['SEED'].astype(float)
cbb['P7'] = cbb['P7'].astype(float)
cbb['DEEP'] = cbb['DEEP'].astype(float)

cbb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 680 entries, 0 to 3227
Data columns (total 21 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   W        680 non-null    float64
 1   ADJOE    680 non-null    float64
 2   ADJDE    680 non-null    float64
 3   BARTHAG  680 non-null    float64
 4   EFG_O    680 non-null    float64
 5   EFG_D    680 non-null    float64
 6   TOR      680 non-null    float64
 7   TORD     680 non-null    float64
 8   ORB      680 non-null    float64
 9   DRB      680 non-null    float64
 10  FTR      680 non-null    float64
 11  FTRD     680 non-null    float64
 12  2P_O     680 non-null    float64
 13  2P_D     680 non-null    float64
 14  3P_O     680 non-null    float64
 15  3P_D     680 non-null    float64
 16  ADJ_T    680 non-null    float64
 17  WAB      680 non-null    float64
 18  SEED     680 non-null    float64
 19  P7       680 non-null    float64
 20  DEEP     680 non-null    float64
dtypes: float64(21)
memor

Now, in summary, I have tidied up this dataset by quantifying categorical variables, accounting for NAs, removing redundant or unnecessary features, and selecting a target feature to analyze in DEEP. In my next report, I will select a model to move forward with. Based on my experiences in class, I plan to try and utilize methods learned in the classification section. I also plan on applying Stratified Splits to my modeling, as there are very few teams that will meet the criteria to have gone deep (see below).

In [10]:
cbb['DEEP'].value_counts()

,count
DEEP,
0.0,600
1.0,80


In [12]:
cbb.to_csv('cbb_cleaned.csv', index=False)